In [1]:
import math
import import_ipynb
import sys
sys.path.append('../')  # Go up two folders to the project root

import structures.DocumentIndex as doc_ind
import structures.LexiconRow as lex_row
import building_data_structures.CollectionStatistics as c_s

importing Jupyter notebook from C:\Users\Davide\IR\Progetto\query_processing\..\structures\DocumentIndex.ipynb
importing Jupyter notebook from C:\Users\Davide\IR\Progetto\query_processing\..\utilities\General_Utilities.ipynb
importing Jupyter notebook from C:\Users\Davide\IR\Progetto\query_processing\..\structures\DocumentIndexRow.ipynb
importing Jupyter notebook from C:\Users\Davide\IR\Progetto\query_processing\..\structures\LexiconRow.ipynb
importing Jupyter notebook from C:\Users\Davide\IR\Progetto\query_processing\..\building_data_structures\CollectionStatistics.ipynb


In [3]:
class Scoring:
    upper_bound_TF: float
    upper_bound_DL: float
        
    collection_statistics:c_s
    
    def __init__(self,collectionStatistic:c_s):
        self.collection_statistics=collectionStatistic

    # quindi riceve la scelta e un oggetto Postin (modifica doc_id e term_freq accorpandoli dentro un oggetto Posting
    def choose_scoring_function(self, choice: str, doc_id: int, term_freq:int):
        return self.compute_BM25_term(doc_id, term_freq) if choice == "bm25" else self.compute_TFIDF()
    
    def compute_BM25_term(self, doc_id: int, term_freq:int, k1:float = 1.6, b:float = 0.75)-> float:
        if not isinstance(doc_id, int) or not isinstance(term_freq, int):
            raise ValueError("Invalid parameters.") 
            
        if doc_id < 0 or term_freq <= 0:
            raise ValueError("doc_id and term_freq must be positive")

        self.c_s.collection_statistics.read_binary_mode()
        n_doc = self.collection_statistics.num_documents
        
        idf = self.compute_IDFT(term_freq)
        avgDL = lex_row.compute_avgDL(doc_index) # sistema questa (doc_index)
        log_tf = (1 + math.log(term_freq))
        doc_len = doc_index.get_document(doc_id).document_length # sistema questa (doc_index)
    
        return (idf * log_tf)/(log_tf + k1 * ( (1 - b) + b * (doc_len/avgDL) ))

    def compute_IDFT(self, dft:int) -> float:
        """
        Compute the inverse document frequency for a term based on the document index (doc_index) and document frequency (dft).
    
        Args:
            dft: The document frequency of the term.
        """
        if dft <= 0:
            raise ValueError("Invalid parameters.")

        
        number_of_documents = self.collection_statistics.num_documents
        
        return math.log10(number_of_documents/dft)  


    def compute_TFIDF(self, tf: int, idf: float) -> float:
        """
        Compute the TF-IDF value based on the term frequency (tf) and inverse document frequency (idf).
        
        Args:
            tf: An integer representing the term frequency.
            idf: A float representing the inverse document frequency.
        """
        if not isinstance(tf, int) or not isinstance(idf, float):
            raise ValueError("Invalid parameters.")
    
        if tf <= 0:
            return 0
            
        return (1 + math.log10(tf)) * idf

    
    '''
    def compute_BM25_query(query:str, doc_id: int, idf: float) -> float:
        bm25 = 0.0
        
        # remove duplicates
        tokens = list(set(query.split()))
        
        for token in tokens:
            #term_freq =  come si trova la term frequencies? bisogna recuperare la postings list dal doc_index?
            bm25 = bm25 + compute_BM25_term(doc_index, doc_id, term_freq)
    
        return bm25
    '''